In [2]:
import pandas as pd
df=pd.read_csv('mental_disorders_reddit.csv')

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import re
import string

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [4]:
df=df.dropna(how='any')

In [5]:
import nltk
!pip install nltk

In [6]:
def convert_lowercase(text):
    text = text.lower()
    return text

def remove_url(text):
    re_url = re.compile('https?://\S+|www\.\S+')
    return re_url.sub('', text)

exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

def remove_stopwords(text):
    new_list = []
    words = word_tokenize(text)
    stopwrds = stopwords.words('english')
    for word in words:
        if word not in stopwrds:
            new_list.append(word)
    return ' '.join(new_list)


def perform_stemming(text):
    stemmer = PorterStemmer()
    new_list = []
    words = word_tokenize(text)
    for word in words:
        new_list.append(stemmer.stem(word))

    return " ".join(new_list)


def perform_lemmatize(text):
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    new_list = []
    words = word_tokenize(text)
    for word in words:
        new_list.append(lem.lemmatize(word))

    return " ".join(new_list)


def category_equalization(data_df,col):# функция для баланса значений категорий в таргет 
    for value in list(data_df[col].unique()):
        
        num_rows_to_remove=data_df[col].value_counts()[value]-data_df[col].value_counts()['schizophrenia']#надо поменять на мимнимальное значение в value_counts 
        
        rows_with_value = data_df[data_df[col] == value]

        data_df = data_df[~data_df.isin(rows_with_value.head(num_rows_to_remove))]
    return data_df

In [7]:
df_equals=category_equalization(df,"subreddit")
df_equals=df_equals.dropna()
df_equals=df_equals[df_equals['selftext'] != 'removed']

In [8]:
df_equals['selftext']=df_equals['selftext'].apply(remove_url)
df_equals['selftext']=df_equals['selftext'].apply(remove_punc)
df_equals['selftext']=df_equals['selftext'].apply(convert_lowercase)
df_equals['selftext']=df_equals['selftext'].apply(remove_stopwords)
df_equals['selftext']=df_equals['selftext'].apply(perform_lemmatize)

df_equals['title']=df_equals['title'].apply(remove_url)
df_equals['title']=df_equals['title'].apply(remove_punc)
df_equals['title']=df_equals['title'].apply(convert_lowercase)
df_equals['title']=df_equals['title'].apply(remove_stopwords)
df_equals['title']=df_equals['title'].apply(perform_lemmatize)

In [9]:
df_equals["subreddit"].value_counts()

BPD              20280
bipolar          20280
depression       20280
Anxiety          20280
schizophrenia    20280
mentalillness    20280
Name: subreddit, dtype: int64

In [10]:
df_equals = df_equals[(df_equals['selftext'].str.len() > 10)]
df_equals=category_equalization(df_equals,'subreddit').dropna(how='any')


In [11]:
df_equals["subreddit"].value_counts()

BPD              11374
bipolar          11374
depression       11374
Anxiety          11374
schizophrenia    11374
mentalillness    11374
Name: subreddit, dtype: int64

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
import numpy as np

РАБОТА С ОДНИМ ПРИЗНАКОМ 'selftext'

In [13]:

y = df_equals['subreddit']
X = df_equals['selftext']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [14]:

pipe_logreg = Pipeline([
           ('vect', TfidfVectorizer()),
           ('clf', LogisticRegression(n_jobs=-1)),
])
parameters = [{
    'vect__max_features':[5000, 10000, 50000],
     'vect__ngram_range': [(1,1), (1,2), (1,3)],
    'clf__penalty': ('l1','l2'),
    'clf__C': (10, 1.0, 0.1, 0.01)
}]

grid_search = GridSearchCV(pipe_logreg, parameters,verbose=5,n_jobs=-1,cv=2)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 2 folds for each of 72 candidates, totalling 144 fits
{'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__max_features': 50000, 'vect__ngram_range': (1, 3)}
0.6580456134900076


In [15]:

pipe_log = Pipeline([
           ('vect', TfidfVectorizer(max_features= 50000,ngram_range=(1,3))),
           ('clf', LogisticRegression(C=1.0,penalty='l2',random_state=42,n_jobs=-1)),
])
pipe_log.fit(X_train, y_train)
y_pred = pipe_log.predict(X_test)
predicted_prob = pipe_log.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6746281778884899
               precision    recall  f1-score   support

      Anxiety       0.75      0.76      0.75      2307
          BPD       0.74      0.74      0.74      2244
      bipolar       0.72      0.67      0.69      2239
   depression       0.58      0.71      0.64      2276
mentalillness       0.55      0.46      0.50      2281
schizophrenia       0.71      0.71      0.71      2302

     accuracy                           0.67     13649
    macro avg       0.68      0.67      0.67     13649
 weighted avg       0.67      0.67      0.67     13649



In [16]:
pipe_cat = Pipeline([
           ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1,3))),
           ('clf', CatBoostClassifier(task_type = 'GPU', verbose=150)),
])
pipe_cat.fit(X_train, y_train)
y_pred = pipe_cat.predict(X_test)
predicted_prob = pipe_cat.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Learning rate set to 0.146779
0:	learn: 1.6484492	total: 427ms	remaining: 7m 6s
150:	learn: 0.9842761	total: 43.2s	remaining: 4m 2s
300:	learn: 0.9097480	total: 1m 22s	remaining: 3m 10s
450:	learn: 0.8769443	total: 1m 57s	remaining: 2m 22s
600:	learn: 0.8563338	total: 2m 31s	remaining: 1m 40s
750:	learn: 0.8398740	total: 3m 4s	remaining: 1m 1s
900:	learn: 0.8266160	total: 3m 38s	remaining: 24s
999:	learn: 0.8188528	total: 3m 59s	remaining: 0us
accuracy %s 0.6670818374972526
               precision    recall  f1-score   support

      Anxiety       0.76      0.74      0.75      2307
          BPD       0.75      0.73      0.74      2244
      bipolar       0.75      0.66      0.70      2239
   depression       0.57      0.70      0.63      2276
mentalillness       0.56      0.44      0.49      2281
schizophrenia       0.64      0.74      0.69      2302

     accuracy                           0.67     13649
    macro avg       0.67      0.67      0.67     13649
 weighted avg       0.67

In [17]:
from lightgbm import LGBMClassifier

In [18]:
pipe_cat = Pipeline([
           ('vect', TfidfVectorizer(max_features=10000,ngram_range=(1,3))),
           ('clf', LGBMClassifier(boosting_type='gbdt', num_leaves=100, max_depth=-1, learning_rate=0.1, n_estimators=200,n_jobs=-1)),
])
pipe_cat.fit(X_train, y_train)
y_pred = pipe_cat.predict(X_test)
predicted_prob = pipe_cat.predict_proba(X_test)

print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6767528756685471
               precision    recall  f1-score   support

      Anxiety       0.73      0.75      0.74      2307
          BPD       0.77      0.75      0.76      2244
      bipolar       0.72      0.67      0.70      2239
   depression       0.59      0.69      0.63      2276
mentalillness       0.54      0.48      0.51      2281
schizophrenia       0.71      0.72      0.71      2302

     accuracy                           0.68     13649
    macro avg       0.68      0.68      0.68     13649
 weighted avg       0.68      0.68      0.68     13649



РАБОТА С ДВУМЯ ПРИЗНАКОМ 'selftext' и 'title'

In [73]:

y = df_equals['subreddit']
le = LabelEncoder()
y = le.fit_transform(y)
X = df_equals[['title','selftext']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42)

In [20]:
from sklearn.pipeline import FeatureUnion
import xgboost as xgb

In [21]:
preprocessor1 = ColumnTransformer(
    transformers=[
        ('tfidf1', TfidfVectorizer(), 'title')
    ])

preprocessor2 = ColumnTransformer(
    transformers=[
        ('tfidf2', TfidfVectorizer(), 'selftext')
    ])

preprocessor = FeatureUnion([
    ('preprocessor1', preprocessor1),
    ('preprocessor2', preprocessor2)
])


model = xgb.XGBClassifier()
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

In [22]:
model = xgb.XGBClassifier(max_depth=100,n_jods=-1)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
predicted_prob = pipeline.predict_proba(X_test)
print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6918431685981973
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      3797
           1       0.78      0.74      0.76      3712
           2       0.78      0.70      0.73      3711
           3       0.59      0.73      0.65      3786
           4       0.56      0.47      0.51      3758
           5       0.69      0.76      0.72      3757

    accuracy                           0.69     22521
   macro avg       0.70      0.69      0.69     22521
weighted avg       0.69      0.69      0.69     22521



In [23]:
model=CatBoostClassifier(task_type = 'GPU', verbose=150)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
predicted_prob = pipeline.predict_proba(X_test)
print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy %s 0.6918431685981973
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      3797
           1       0.78      0.74      0.76      3712
           2       0.78      0.70      0.73      3711
           3       0.59      0.73      0.65      3786
           4       0.56      0.47      0.51      3758
           5       0.69      0.76      0.72      3757

    accuracy                           0.69     22521
   macro avg       0.70      0.69      0.69     22521
weighted avg       0.69      0.69      0.69     22521



In [75]:
from sklearn.neural_network import MLPClassifier

model=MLPClassifier(hidden_layer_sizes=200,activation='tanh',solver='sgd')
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
predicted_prob = pipeline.predict_proba(X_test)
print('accuracy %s',  accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))


accuracy %s 0.6918431685981973
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      3797
           1       0.78      0.74      0.76      3712
           2       0.78      0.70      0.73      3711
           3       0.59      0.73      0.65      3786
           4       0.56      0.47      0.51      3758
           5       0.69      0.76      0.72      3757

    accuracy                           0.69     22521
   macro avg       0.70      0.69      0.69     22521
weighted avg       0.69      0.69      0.69     22521



In [ ]:
tfidf_param_grid = {
    'preprocessor__preprocessor1__tfidf1__max_features': [1000,5000,10000],
    'preprocessor__preprocessor1__tfidf1__ngram_range': [(1, 1), (1, 3)],
    'preprocessor__preprocessor2__tfidf2__max_features': [2500, 1000,5000],
    'preprocessor__preprocessor2__tfidf2__ngram_range': [(1, 1), (1, 3)],
}

# Define the hyperparameters for XGBoost
xgb_param_grid = {
    'model__learning_rate': [0.05],
    'model__max_depth': [3, 5, 10, 20],
    'model__n_estimators': [250,500,1000]
}

param_grid = {**tfidf_param_grid, **xgb_param_grid}

In [ ]:
model = xgb.XGBClassifier()
grid_search = GridSearchCV(pipeline, param_grid, verbose=5, n_jobs=-1, cv=2)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

Fitting 2 folds for each of 432 candidates, totalling 864 fits


In [24]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [50]:
y = df_equals['subreddit']
le = LabelEncoder()
y = le.fit_transform(y)
X = df_equals[['selftext','subreddit']]

In [51]:
X

,selftext,subreddit
226438,one persistent difficult symptom bpd impulsive...,BPD
226439,ive started dbt programme ive taking medicatio...,BPD
226440,trapped couch dissociating something want get ...,BPD
226441,upside bpd terminal,BPD
226442,give example let try make light make humorous ...,BPD
...,...,...
701778,pattern loosely related random word sentence t...,mentalillness
701779,cant afford real session 11 pm somebody please...,mentalillness
701781,hello im taking step get rid pica ive picaing ...,mentalillness
701782,someone war veteran know mentally ill wont see...,mentalillness
